In [1]:
import os
import pandas as pd
import sys
    
from dbfread import DBF

pathWebData = r'E:\WebAppBuilderForArcGIS_v2.20\server\apps\8\widgets\tttScenarioManager\data'


In [2]:
_reImportBQ = False

if _reImportBQ: 

    sys.path.insert(0, '..\..\Resources\Python-Functions')
    import BigQuery

    # import transit route outputs
    bqclient = BigQuery.getBigQueryClient_TDMScenarios()

    dfScenarios = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios").to_dataframe()
    #dfScenarios
    dfScenarios.to_csv("_dfScenarios.csv", index=False)

    dfODDetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_od_detail").to_dataframe()
    dfODDetail.to_csv("_dfODDetail.csv", index=False)
    #display(dfODDetail)

    dfPADetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_pa_detail").to_dataframe()
    dfPADetail.to_csv("_dfPADetail.csv", index=False)
    #display(dfPADetail)
else:
    print ("USING PREVIOUSLY IMPORTED DATA!!!!")
    dfScenarios = pd.read_csv("_dfScenarios.csv")
    print ("USING PREVIOUSLY IMPORTED DATA!!!!")
    dfODDetail = pd.read_csv("_dfODDetail.csv")
    print ("USING PREVIOUSLY IMPORTED DATA!!!!")
    dfPADetail = pd.read_csv("_dfPADetail.csv")

USING PREVIOUSLY IMPORTED DATA!!!!
USING PREVIOUSLY IMPORTED DATA!!!!
USING PREVIOUSLY IMPORTED DATA!!!!


In [3]:
# read in manully created file
dfRouteSummaryNames_Revised = pd.read_csv(r"..\Resources\TDM\RouteRenumbering\manual\routesummarynames_revised.csv")
#dfRouteSummaryNames_Revised

In [4]:
paDropColumns = ['NAME',
                 'LONGNAME'     ,
                 'numRecords'   ,
                 'minYear'      ,
                 'maxYear'      ,
                 'minMode'      ,
                 'NAMEwithZeros',
                 'ROUTEID'      ,
                 'NAMEID'       ,
                 'OPERATOR'     ,
                 'PK_HEADWAY'   ,
                 'OK_HEADWAY'   ,
                 'PK_TIME'      ,
                 'OK_TIME'      ,
                 'DIST'         ,
                 'PK_D_BRDA'    ,
                 'PK_W_BRDA'    ,
                 'PK_BRDA'      ,
                 'PK_XITB'      ,
                 'OK_D_BRDA'    ,
                 'OK_W_BRDA'    ,
                 'OK_BRDA'      ,
                 'OK_XITB'      ,
                 'DY_D4_BRDA'   ,
                 'DY_W4_BRDA'   ,
                 'DY_4_XITB'    ,
                 'DY_D5_BRDA'   ,
                 'DY_W5_BRDA'   ,
                 'DY_5_XITB'    ,
                 'DY_D6_BRDA'   ,
                 'DY_W6_BRDA'   ,
                 'DY_6_XITB'    ,
                 'DY_D7_BRDA'   ,
                 'DY_W7_BRDA'   ,
                 'DY_7_XITB'    ,
                 'DY_D8_BRDA'   ,
                 'DY_W8_BRDA'   ,
                 'DY_8_XITB'    ,
                 'DY_D9_BRDA'   ,
                 'DY_W9_BRDA'   ,
                 'DY_9_XITB'    ,
                 'DY_D_BRDA'    ,
                 'DY_W_BRDA'    ,
                 'DY_BRDA'      ,
                 'DY_XITB'      ]


In [5]:
dfODDetailWithRevNames = pd.DataFrame.merge(dfODDetail,dfRouteSummaryNames_Revised,on='NAME')
dfODDetailWithRevNames = dfODDetailWithRevNames.drop(columns=('NAME'))
dfODDetailWithRevNames = dfODDetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfODDetailWithRevNames)

# change negative to column
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]!='-'), 'Direction'] = 'P'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'Direction'] = 'N'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'NAME'] = dfPADetail['NAME'].str[:-1]

dfPADetailWithRevNames = pd.DataFrame.merge(dfPADetail,dfRouteSummaryNames_Revised,on='NAME')
dfPADetailWithRevNames = dfPADetailWithRevNames.drop(columns=(paDropColumns))
dfPADetailWithRevNames = dfPADetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfPADetailWithRevNames)

In [6]:
dfTotVol = pd.DataFrame()

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    dfTemp = dfODDetailWithRevNames[dfODDetailWithRevNames['scenarioID']==row['scenarioID']]
    #display(dfTemp)

    dfLinkLookup = dfTemp[['scenarioID','MODE','NAMEID','NAME','LINKSEQ1','A']].copy()
    dfLinkLookup['LINKSEQ_OF_B'] = dfLinkLookup['LINKSEQ1'] + 1

    dfLinkLookupMax = dfLinkLookup.groupby(['scenarioID','NAMEID','NAME'],as_index=False).agg(MAXLINKSEQ1=('LINKSEQ1','max'))
    dfLinkLookupMax['MAXLINKSEQp1'] = dfLinkLookupMax['MAXLINKSEQ1'] + 1
    dfLinkLookupWithMaxPlus1 = pd.DataFrame.merge(dfLinkLookup, dfLinkLookupMax, on=('scenarioID','NAMEID','NAME'), how='left')
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ1'    ]==0), 'LINKSEQ1'    ] = dfLinkLookupWithMaxPlus1['MAXLINKSEQp1']
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ_OF_B']==1), 'LINKSEQ_OF_B'] = 0

    dfLinkLookupWithMaxPlus1 = dfLinkLookupWithMaxPlus1.drop(columns=['MAXLINKSEQ1','MAXLINKSEQp1'])

    dfJoinToGetB = pd.DataFrame.merge(dfLinkLookupWithMaxPlus1, dfLinkLookupWithMaxPlus1, left_on=('MODE','NAMEID','NAME','LINKSEQ_OF_B'), right_on=('MODE','NAMEID','NAME','LINKSEQ1'))
    dfJoinToGetB = dfJoinToGetB[['MODE','NAMEID','NAME','LINKSEQ1_x','A_x','A_y']]
    dfJoinToGetB.columns = ('MODE','NAMEID','NAME','LINKSEQ1','A','B')

    dfTempPrepped = pd.DataFrame.merge(dfJoinToGetB,dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A','DIST','D_TOT_VOL','scenarioID']], on=('MODE','NAMEID','NAME','LINKSEQ1','A'))

    #display(dfLinkLookup)
    #display(dfLinkLookupMax)
    #display(dfLinkLookupWithMaxPlus1)
    #display(dfJoinToGetB)
    #display(dfTempPrepped)

    dfTotVol = dfTotVol.append(dfTempPrepped)

    #display(dfLinkLookup.sort_values(by='LINK', ascending=False).drop_duplicates(subset='NAME'))

display(dfTotVol)
display(dfTotVol['D_TOT_VOL'].sum())

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID
0,4,9.0,UT805,1,67413,67421,0.09,0.08,3.05
1,4,9.0,UT805,39,66790,71488,0.14,0.06,3.05
2,4,9.0,UT805,41,66825,66826,0.10,0.02,3.05
3,4,9.0,UT805,44,50024,66841,0.10,19.84,3.05
4,4,9.0,UT805,47,66825,71488,0.14,21.25,3.05
...,...,...,...,...,...,...,...,...,...
6644,4,16.0,UT809F,94,68565,68576,0.12,11.52,6.18
6645,4,16.0,UT809F,95,68576,68608,0.10,11.52,6.18
6646,4,16.0,UT809F,96,68608,68651,0.10,11.52,6.18
6647,4,16.0,UT809F,97,68651,68670,0.09,11.52,6.18


258572664.49999997

In [7]:
# Add SegIDs
dfNetWithSegIDs = pd.DataFrame(DBF(r"E:\GitHub\WF-TDM-v9.0\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-03-02 - Link.DBF", load=True))
#dfNetWithSegIDs = pd.DataFrame(DBF("\\\modelace\\ModelAce-E\\1 - TDM\\2 - Sandbox\\v9_Development\\WF TDM v9.0 - 2022-12-19\\Scenarios\\Needs_2050\\0_InputProcessing\\v9_Needs_SE50_Net50_Link.DBF", load=True))
#dfNetWithSegIDs = dfNetWithSegIDs[['A','B','SEGID']]
dfNetWithSegIDs['IB_OB'] = 'XX' # IB / OB NOT IN MASTER NETWORK!!!
dfNetWithSegIDs = dfNetWithSegIDs[['A','B','SEGID','IB_OB']]
#dfNetWithSegIDs = dfNetWithSegIDs[dfNetWithSegIDs['SEGID']!='']
dfNetWithSegIDs

,A,B,SEGID,IB_OB
0,1,29685,,XX
1,2,29685,,XX
2,3,29645,,XX
3,4,29671,,XX
4,5,29680,,XX
...,...,...,...,...
56446,95064,95025,,XX
56447,95064,95026,,XX
56448,95064,95056,,XX
56449,95065,3621,,XX


In [8]:
# join od detail with link seg data. only preserve rows with data in both tables since some links don't have segids etc
dfTotVolWithSegIDs = pd.DataFrame.merge(dfTotVol,dfNetWithSegIDs,on=('A','B'),how='inner')
dfTotVolWithSegIDs['DISTxVOL'] = dfTotVolWithSegIDs['DIST'] * dfTotVolWithSegIDs['D_TOT_VOL']
dfTotVolWithSegIDs

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
0,4,9.0,UT805,1,67413,67421,0.09,0.08,3.05,2980_000.0,XX,0.0072
1,4,12.0,UT822,150,67413,67421,0.09,78.96,3.05,2980_000.0,XX,7.1064
2,4,9.0,UT805,1,67413,67421,0.09,0.04,3.08,2980_000.0,XX,0.0036
3,4,12.0,UT822,150,67413,67421,0.09,22.87,3.08,2980_000.0,XX,2.0583
4,4,1.0,UT805,1,67413,67421,0.09,0.28,3.11,2980_000.0,XX,0.0252
...,...,...,...,...,...,...,...,...,...,...,...,...
355399,4,35.0,MAG_023,106,69316,69314,0.07,12.25,2.13,MAG_6140,XX,0.8575
355400,9,92.0,UT830X,138,67502,67511,0.07,786.96,2.07,,XX,55.0872
355401,4,31.0,MAG_026,26,65922,65923,0.14,91.07,2.15,,XX,12.7498
355402,4,31.0,MAG_026,27,65923,65926,0.20,142.49,2.15,,XX,28.4980


In [39]:
dfTotVolWithSegIDs['scenarioID'].unique()

array([3.05, 3.08, 3.11, 3.16, 4.16, 5.16, 3.01, 4.01, 5.01, 3.02, 4.02,
       5.02, 3.18, 4.18, 5.18, 3.03, 3.06, 3.09, 3.13, 4.13, 5.13, 3.04,
       3.07, 3.1 , 3.15, 4.15, 5.15, 2.08, 2.11, 2.01, 2.02, 2.18, 2.06,
       2.09, 2.07, 2.1 , 6.01, 6.02, 6.13, 6.15, 6.16, 6.18, 2.16, 2.13,
       2.15, 1.16, 1.01, 1.17, 1.12, 1.14])

In [42]:
dfTotVolWithSegIDs[(dfTotVolWithSegIDs['scenarioID']==1.01)]

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
264704,9,109.0,UT830X,51,22939,22910,0.06,3124.30,1.01,0048_004.1,XX,187.4580
295423,4,13.0,UT834,37,23870,23869,0.24,128.85,1.01,2218_000.5,XX,30.9240
351585,7,106.0,703_Red,12,20063,20062,0.61,13441.53,1.01,2153_002.1,XX,8199.3333
351592,7,106.0,703_Red,26,20013,20017,0.27,7115.52,1.01,2030_002.7,XX,1921.1904
351597,8,108.0,750_FR,35,20076,20075,2.39,7187.13,1.01,2187_000.4,XX,17177.2407
351602,4,16.0,UT862,47,22540,22541,0.28,201.12,1.01,2080_005.7,XX,56.3136
355240,7,104.0,701_Blue,7,20021,20010,0.95,3133.54,1.01,2031_002.2,XX,2976.8630
355244,7,104.0,701_Blue,9,20020,20019,0.68,3707.31,1.01,2030_005.1,XX,2520.9708
355248,7,104.0,701_Blue,10,20019,20018,0.99,4480.69,1.01,2030_005.1,XX,4435.8831
355252,7,104.0,701_Blue,12,20017,20013,0.27,5434.20,1.01,2030_002.7,XX,1467.2340


In [46]:
dfTotVolWithSegIDs[(dfTotVolWithSegIDs['scenarioID']==1.01)]

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
264704,9,109.0,UT830X,51,22939,22910,0.06,3124.30,1.01,0048_004.1,XX,187.4580
295423,4,13.0,UT834,37,23870,23869,0.24,128.85,1.01,2218_000.5,XX,30.9240
351585,7,106.0,703_Red,12,20063,20062,0.61,13441.53,1.01,2153_002.1,XX,8199.3333
351592,7,106.0,703_Red,26,20013,20017,0.27,7115.52,1.01,2030_002.7,XX,1921.1904
351597,8,108.0,750_FR,35,20076,20075,2.39,7187.13,1.01,2187_000.4,XX,17177.2407
351602,4,16.0,UT862,47,22540,22541,0.28,201.12,1.01,2080_005.7,XX,56.3136
355240,7,104.0,701_Blue,7,20021,20010,0.95,3133.54,1.01,2031_002.2,XX,2976.8630
355244,7,104.0,701_Blue,9,20020,20019,0.68,3707.31,1.01,2030_005.1,XX,2520.9708
355248,7,104.0,701_Blue,10,20019,20018,0.99,4480.69,1.01,2030_005.1,XX,4435.8831
355252,7,104.0,701_Blue,12,20017,20013,0.27,5434.20,1.01,2030_002.7,XX,1467.2340


In [48]:
dfTemp = dfTotVolWithSegIDs[(dfTotVolWithSegIDs['SEGID']=='0089_378.6')]
dfTemp

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
229587,5,74.0,WFRC_030,19,25440,25464,0.08,4047.86,3.05,0089_378.6,XX,323.8288
229588,5,74.0,WFRC_030,19,25440,25464,0.08,3025.13,3.08,0089_378.6,XX,242.0104
229589,5,74.0,WFRC_030,19,25440,25464,0.08,4154.47,3.16,0089_378.6,XX,332.3576
229590,5,68.0,WFRC_030,19,25440,25464,0.08,3638.04,4.16,0089_378.6,XX,291.0432
229591,5,68.0,WFRC_030,19,25440,25464,0.08,3539.01,5.16,0089_378.6,XX,283.1208
...,...,...,...,...,...,...,...,...,...,...,...,...
346349,7,76.0,703_Red,18,15171,15170,0.08,13750.03,6.02,0089_378.6,XX,1100.0024
346350,7,77.0,703_Red,18,15171,15170,0.08,12066.55,6.13,0089_378.6,XX,965.3240
346351,7,82.0,703_Red,18,15171,15170,0.08,14268.32,6.15,0089_378.6,XX,1141.4656
346352,7,91.0,703_Red,18,15171,15170,0.08,16429.43,6.16,0089_378.6,XX,1314.3544


In [10]:
dfTotVolStatsBySegID = dfTotVolWithSegIDs.groupby(['scenarioID','MODE','NAME','SEGID'],as_index=False).agg(DISTxVOL=('DISTxVOL','sum'),DIST=('DIST','sum'))
dfTotVolStatsBySegID['SEGVOL'] = round(dfTotVolStatsBySegID['DISTxVOL'] / dfTotVolStatsBySegID['DIST'],0).astype(int)
dfTotVolStatsBySegID.drop(columns=['DISTxVOL','DIST'],inplace=True)
dfTotVolStatsBySegID

,scenarioID,MODE,NAME,SEGID,SEGVOL
0,1.01,4,UT806,WFRC_8222,58
1,1.01,4,UT834,2218_000.5,129
2,1.01,4,UT862,2080_005.7,201
3,1.01,7,701_Blue,2030_002.7,5434
4,1.01,7,701_Blue,2030_004.6,7362
...,...,...,...,...,...
106890,6.18,9,WE603,3400_000.1,1131
106891,6.18,9,WE603,3400_000.3,1161
106892,6.18,9,WE603,3400_000.4,1445
106893,6.18,9,WE603,3400_000.6,1445


In [44]:
dfTotVolStatsBySegID[dfTotVolStatsBySegID['SEGID']=='0089_378.6']

,scenarioID,MODE,NAME,SEGID,SEGVOL
114,2.01,4,DA460,0089_378.6,8
146,2.01,4,DA461,0089_378.6,22
184,2.01,4,DA462,0089_378.6,10
1241,2.01,4,SL463,0089_378.6,18
1363,2.01,4,SL471,0089_378.6,22
...,...,...,...,...,...
102321,6.15,7,WFRC_055,0089_378.6,3344
104174,6.16,5,WFRC_030,0089_378.6,3092
104642,6.16,7,703_Red,0089_378.6,16429
104708,6.16,7,WFRC_055,0089_378.6,7684


In [11]:
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID.groupby(['scenarioID','SEGID','MODE'], as_index=False).agg(SEGVOLBYMODE=('SEGVOL','sum'))
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.pivot_table(index=('scenarioID','SEGID'), columns='MODE')
dfTotVolStatsBySegID_byMode.fillna(0,inplace=True)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.astype(int)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.reset_index()
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.droplevel(0, axis=1) 
dfTotVolStatsBySegID_byMode.columns = ['scenarioID','SEGID','M4','M5','M6','M7','M8','M9']
dfTotVolStatsBySegID_byMode['MT'] =  dfTotVolStatsBySegID_byMode['M4'] + \
                                     dfTotVolStatsBySegID_byMode['M5'] + \
                                     dfTotVolStatsBySegID_byMode['M6'] + \
                                     dfTotVolStatsBySegID_byMode['M7'] + \
                                     dfTotVolStatsBySegID_byMode['M8'] + \
                                     dfTotVolStatsBySegID_byMode['M9']
dfTotVolStatsBySegID_byMode

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
0,1.01,0048_004.1,0,0,0,0,0,3124,3124
1,1.01,2030_002.7,0,0,0,12550,0,0,12550
2,1.01,2030_004.6,0,0,0,14565,0,0,14565
3,1.01,2030_005.1,0,0,0,4166,0,0,4166
4,1.01,2031_002.2,0,0,0,3134,0,0,3134
...,...,...,...,...,...,...,...,...,...
65580,6.18,WFRC_8227,583,0,0,0,0,0,583
65581,6.18,WFRC_8256,25,0,0,0,0,0,25
65582,6.18,WFRC_8258,91,0,0,0,0,0,91
65583,6.18,WFRC_8260,107,0,0,0,0,0,107


In [12]:
df = dfTotVolStatsBySegID_byMode

df

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
0,1.01,0048_004.1,0,0,0,0,0,3124,3124
1,1.01,2030_002.7,0,0,0,12550,0,0,12550
2,1.01,2030_004.6,0,0,0,14565,0,0,14565
3,1.01,2030_005.1,0,0,0,4166,0,0,4166
4,1.01,2031_002.2,0,0,0,3134,0,0,3134
...,...,...,...,...,...,...,...,...,...
65580,6.18,WFRC_8227,583,0,0,0,0,0,583
65581,6.18,WFRC_8256,25,0,0,0,0,0,25
65582,6.18,WFRC_8258,91,0,0,0,0,0,91
65583,6.18,WFRC_8260,107,0,0,0,0,0,107


In [13]:
dfScenarios.sort_values('scenarioID')

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
31,1.01,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019
36,1.12,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2030,RTP,2030
40,1.14,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2040,RTP,2040
27,1.16,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2050,RTP,2050
34,1.17,v832,WF TDM v8.3.2 - 2022-02-04a,TIP_2024,TIP,2024
32,2.01,v832,WF TDM v8.3.2 - 2022-09-21,BY_2019,BY,2019
33,2.02,v832,WF TDM v8.3.2 - 2022-09-21,BY_2023,BY,2023
37,2.06,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2032,Needs,2032
41,2.07,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2042,Needs,2042
28,2.08,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2050,Needs,2050


In [14]:
# read in json ids
dfWABScenarios = pd.read_json(r"E:\WebAppBuilderForArcGIS_v2.20\server\apps\8\widgets\tttScenarioManager\data\scenarios.json")
#display(dfWABScenarios)

# match with BigQuery Scenarios

dfBigQueryToWAB = pd.DataFrame([
    [1.01, 'v8.3.2 BY 2019'              ],
    [1.12, 'v8.3.2 RTP 2030'             ],
    [1.14, 'v8.3.2 RTP 2040'             ],
    [1.16, 'v8.3.2 RTP 2050'             ],
    [1.17, 'v8.3.2 TIP 2024'             ],
    [3.01, 'v9 Beta BY 2019'             ],
    [3.02, 'v9 Beta BY 2023'             ],
    [3.03, 'v9 Beta RTP+EF 2032'         ],
    [3.04, 'v9 Beta RTP+EF 2042'         ],
    [3.05, 'v9 Beta RTP+EF 2050'         ],
    [3.06, 'v9 Beta Needs 2032'          ],
    [3.07, 'v9 Beta Needs 2042'          ],
    [3.08, 'v9 Beta Needs 2050'          ],
    [3.09, 'v9 Beta No Build 2032'       ],
    [3.10, 'v9 Beta No Build 2042'       ],
    [3.11, 'v9 Beta No Build 2050'       ],
    [3.13, 'v9 Beta RTP 2032'            ],
    [3.15, 'v9 Beta RTP 2042'            ],
    [3.16, 'v9 Beta RTP 2050'            ],
    [3.18, 'v9 Beta TIP 2028'            ],
    [4.01, 'v9 Beta MC Calib-r0 BY 2019' ],
    [4.02, 'v9 Beta MC Calib-r0 BY 2023' ],
    [4.18, 'v9 Beta MC Calib-r0 TIP 2028'],
    [4.13, 'v9 Beta MC Calib-r0 RTP 2032'],
    [4.15, 'v9 Beta MC Calib-r0 RTP 2042'],
    [4.16, 'v9 Beta MC Calib-r0 RTP 2050'],
    [5.01, 'v9 Beta MC Calib-r2 BY 2019' ],
    [5.02, 'v9 Beta MC Calib-r2 BY 2023' ],
    [5.18, 'v9 Beta MC Calib-r2 TIP 2028'],
    [5.13, 'v9 Beta MC Calib-r2 RTP 2032'],
    [5.15, 'v9 Beta MC Calib-r2 RTP 2042'],
    [5.16, 'v9 Beta MC Calib-r2 RTP 2050'],
    [6.01, 'v8.3.2 2023-02-28 BY 2019'   ],
    [6.02, 'v8.3.2 2023-02-28 BY 2023'   ],
    [6.18, 'v8.3.2 2023-02-28 TIP 2028'  ],
    [6.13, 'v8.3.2 2023-02-28 RTP 2032'  ],
    [6.15, 'v8.3.2 2023-02-28 RTP 2042'  ],
    [6.16, 'v8.3.2 2023-02-28 RTP 2050'  ]
],columns=('scenarioID', 'label'))
#display(dfBigQueryToWAB)

dfJoin = pd.DataFrame.merge(dfBigQueryToWAB,dfWABScenarios,on='label',how='left')
dfJoin = dfJoin[['scenarioID','value']]
dfJoin

,scenarioID,value
0,1.01,id_12066090786226718a9b4755.54851854
1,1.12,id_602780967622672681e6327.84737895
2,1.14,id_4927651456226727eb56079.08455813
3,1.16,id_16231621306226729673c147.07236076
4,1.17,id_3989814362267228a9f4b5.00876734
5,3.01,id_37001244863cadd40e1b8f7.52642706
6,3.02,id_72534583163cb54226ea202.98630416
7,3.03,id_73730688563cb5d2222cdc4.25550859
8,3.04,id_71963308763cb5d35348012.40299726
9,3.05,id_23888985563cb5d49295ba5.99558740


In [15]:
dfJoin[dfJoin['scenarioID']==2].value

Series([], Name: value, dtype: object)

In [28]:
export = True

_outputdir_mode = os.path.join(pathWebData,'transitdetailbymode')
_outputdir_route = os.path.join(pathWebData,'transitdetailbyroute')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        #print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
        #print(_id)
        _file = _id + '.json'
        #print(_file)
        _outputfile_mode = os.path.join(_outputdir_mode,_file)
        _outputfile_route = os.path.join(_outputdir_route,_file)

        dfModeOut = dfTotVolStatsBySegID_byMode[dfTotVolStatsBySegID_byMode['scenarioID']==row['scenarioID']]
        dfModeOut = dfModeOut.drop(columns=('scenarioID'))
        dfModeOut = dfModeOut.set_index('SEGID')
        if export: dfModeOut.to_json(_outputfile_mode,orient="table", indent=1) #,orient='records'

        dfRouteOut = dfTotVolStatsBySegID[dfTotVolStatsBySegID['scenarioID']==row['scenarioID']]
        dfRouteOut = dfRouteOut.drop(columns=('scenarioID'))
        dfRouteOut = dfRouteOut.set_index('SEGID')
        if export: dfRouteOut.to_json(_outputfile_route,orient="table", indent=1) #,orient='records'
        #display(dfTemp)
    except:
        print('scenarioID ' + str(row['scenarioID']) + ' error')
        continue

scenarioID 2.08 error
scenarioID 2.11 error
scenarioID 2.16 error
scenarioID 2.01 error
scenarioID 2.02 error
scenarioID 2.18 error
scenarioID 2.06 error
scenarioID 2.09 error
scenarioID 2.13 error
scenarioID 2.07 error
scenarioID 2.1 error
scenarioID 2.15 error


In [19]:
dfPADetail_melt = dfPADetailWithRevNames
dfPADetail_melt = dfPADetail_melt.melt(id_vars=['scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB'], var_name='TransitVariable', value_name='TransitValue')
dfPADetail_melt
#dfPADetail_melt.columns

,scenarioID,NAME,Direction,MODE,LINKSEQ,A,B,STOPA,STOPB,TransitVariable,TransitValue
0,5.01,DA461,P,4,16,26574,26602,1,1,PK_D4_BRDA,0.0
1,5.01,DA461,P,4,17,26602,26616,1,1,PK_D4_BRDA,0.0
2,5.01,DA461,P,4,48,26605,26618,1,1,PK_D4_BRDA,0.0
3,5.01,DA461,P,4,183,26618,26605,1,1,PK_D4_BRDA,0.0
4,5.01,DA461,P,4,214,26616,26602,1,1,PK_D4_BRDA,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20568703,2.13,SL672X,P,6,37,27384,27369,0,1,OK_9_XITB,0.0
20568704,2.15,SL672X,P,6,25,27672,27701,0,1,OK_9_XITB,0.0
20568705,2.16,SL672X,P,6,25,27672,27701,0,1,OK_9_XITB,0.0
20568706,2.16,SL672X,P,6,37,27384,27369,0,1,OK_9_XITB,0.0


In [20]:
dfVarToFields = pd.DataFrame([
 ['PK_D4_BRDA'   , 'PK', 'D', 4, 'BRDA'],
 ['PK_W4_BRDA'   , 'PK', 'W', 4, 'BRDA'],
 ['PK_4_XITB'    , 'PK', 'X', 4, 'XITB'],
 ['PK_D5_BRDA'   , 'PK', 'D', 5, 'BRDA'],
 ['PK_W5_BRDA'   , 'PK', 'W', 5, 'BRDA'],
 ['PK_5_XITB'    , 'PK', 'X', 5, 'XITB'],
 ['PK_D6_BRDA'   , 'PK', 'D', 6, 'BRDA'],
 ['PK_W6_BRDA'   , 'PK', 'W', 6, 'BRDA'],
 ['PK_6_XITB'    , 'PK', 'X', 6, 'XITB'],
 ['PK_D7_BRDA'   , 'PK', 'D', 7, 'BRDA'],
 ['PK_W7_BRDA'   , 'PK', 'W', 7, 'BRDA'],
 ['PK_7_XITB'    , 'PK', 'X', 7, 'XITB'],
 ['PK_D8_BRDA'   , 'PK', 'D', 8, 'BRDA'],
 ['PK_W8_BRDA'   , 'PK', 'W', 8, 'BRDA'],
 ['PK_8_XITB'    , 'PK', 'X', 8, 'XITB'],
 ['PK_D9_BRDA'   , 'PK', 'D', 9, 'BRDA'],
 ['PK_W9_BRDA'   , 'PK', 'W', 9, 'BRDA'],
 ['PK_9_XITB'    , 'PK', 'X', 9, 'XITB'],
 ['OK_D4_BRDA'   , 'OK', 'D', 4, 'BRDA'],
 ['OK_W4_BRDA'   , 'OK', 'W', 4, 'BRDA'],
 ['OK_4_XITB'    , 'OK', 'X', 4, 'XITB'],
 ['OK_D5_BRDA'   , 'OK', 'D', 5, 'BRDA'],
 ['OK_W5_BRDA'   , 'OK', 'W', 5, 'BRDA'],
 ['OK_5_XITB'    , 'OK', 'X', 5, 'XITB'],
 ['OK_D6_BRDA'   , 'OK', 'D', 6, 'BRDA'],
 ['OK_W6_BRDA'   , 'OK', 'W', 6, 'BRDA'],
 ['OK_6_XITB'    , 'OK', 'X', 6, 'XITB'],
 ['OK_D7_BRDA'   , 'OK', 'D', 7, 'BRDA'],
 ['OK_W7_BRDA'   , 'OK', 'W', 7, 'BRDA'],
 ['OK_7_XITB'    , 'OK', 'X', 7, 'XITB'],
 ['OK_D8_BRDA'   , 'OK', 'D', 8, 'BRDA'],
 ['OK_W8_BRDA'   , 'OK', 'W', 8, 'BRDA'],
 ['OK_8_XITB'    , 'OK', 'X', 8, 'XITB'],
 ['OK_D9_BRDA'   , 'OK', 'D', 9, 'BRDA'],
 ['OK_W9_BRDA'   , 'OK', 'W', 9, 'BRDA'],
 ['OK_9_XITB'    , 'OK', 'X', 9, 'XITB']
], columns=('TransitVariable','TimeOfDay','AccessMode','HeirarchyMode','BrdAXitB'))
dfVarToFields

,TransitVariable,TimeOfDay,AccessMode,HeirarchyMode,BrdAXitB
0,PK_D4_BRDA,PK,D,4,BRDA
1,PK_W4_BRDA,PK,W,4,BRDA
2,PK_4_XITB,PK,X,4,XITB
3,PK_D5_BRDA,PK,D,5,BRDA
4,PK_W5_BRDA,PK,W,5,BRDA
5,PK_5_XITB,PK,X,5,XITB
6,PK_D6_BRDA,PK,D,6,BRDA
7,PK_W6_BRDA,PK,W,6,BRDA
8,PK_6_XITB,PK,X,6,XITB
9,PK_D7_BRDA,PK,D,7,BRDA


# Riders by Link

In [21]:
dfPALong = pd.DataFrame.merge(dfPADetail_melt, dfVarToFields, on='TransitVariable')
dfPALong.drop(columns=('TransitVariable'),inplace=True)
dfPALong_pivot = dfPALong.pivot_table(index=('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode'), columns=('AccessMode','BrdAXitB'))
dfPALong_pivot.columns = dfPALong_pivot.columns.to_flat_index()
dfPALong_pivot = dfPALong_pivot.reset_index()

# rename some columns
dfPALong_pivot.columns = ('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode','D_BRDA','W_BRDA','X_XITB')

dfPALong_pivot_withIBOB = pd.DataFrame.merge(dfPALong_pivot,dfNetWithSegIDs,on=('A','B'),how='left')
dfPALong_pivot_withIBOB = dfPALong_pivot_withIBOB.fillna(0)

In [22]:
# CALCULATE RIDERS BY LINK

# column denifinitions
_keyColumns = ['scenarioID','TimeOfDay','MODE','NAME','Direction','HeirarchyMode']
_sortColumns = _keyColumns + ['LINKSEQ']

# sort by for transit direcition
dfPAPivot = dfPALong_pivot_withIBOB.sort_values(by=_sortColumns)

# shift exits to next link in order to facilitate rider calcs
dfPAPivot['X_XITB_Shift'] = dfPAPivot['X_XITB'].shift(1)

# set first link seq to zero because shift would have moved from previous route/time/direction/etc.
dfPAPivot.loc[dfPAPivot['LINKSEQ']==1,'X_XITB_Shift'] = 0

# calculate cummulative sums by key columns
dfPAPivot['D_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['D_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['W_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['W_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['X_XITB_Shift_cumsum'] = dfPAPivot.groupby(_keyColumns)['X_XITB_Shift'].transform(pd.Series.cumsum)

# riders is total of cumulative drive and walk boardings minus the shift in exits
dfPAPivot['Riders'] = dfPAPivot['D_BRDA_cumsum'] + dfPAPivot['W_BRDA_cumsum'] - dfPAPivot['X_XITB_Shift_cumsum']

In [23]:
dfPARiders = dfPAPivot.drop(columns=(['D_BRDA_cumsum','W_BRDA_cumsum','X_XITB_Shift_cumsum']))
dfPARiders['LinkID'] = dfPARiders['A'].astype(str) + '_' + dfPARiders['B'].astype(str)
dfPARiders = dfPARiders[['scenarioID','TimeOfDay','MODE','NAME','HeirarchyMode','LinkID','IB_OB','SEGID','Riders']]
dfPARiders['IB_OB'] = dfPARiders['IB_OB'].fillna('XX')
dfPARiders['SEGID'] = dfPARiders['SEGID'].fillna('')
dfPARiders.columns = ['s','tod','md','nm','hmd','lid','io','segid','r']
dfPARiders

,s,tod,md,nm,hmd,lid,io,segid,r
4584,1.01,OK,4,BE630,4,20142_6885,0,0,13.28
4596,1.01,OK,4,BE630,4,6885_6923,0,0,13.28
4608,1.01,OK,4,BE630,4,6923_6888,0,0,13.29
4620,1.01,OK,4,BE630,4,6888_9357,0,0,13.44
4632,1.01,OK,4,BE630,4,9357_6889,0,0,13.44
...,...,...,...,...,...,...,...,...,...
6841835,6.18,PK,9,WE603,9,28727_28678,XX,0203_002.3,41.68
6841847,6.18,PK,9,WE603,9,28678_28651,XX,0203_001.8,41.14
6841859,6.18,PK,9,WE603,9,28651_28631,XX,0203_001.8,9.72
6841871,6.18,PK,9,WE603,9,28631_28619,XX,0203_001.6,9.71


# Boardings/Alightings by Node

In [24]:
# calculate boardings
dfPABoardings  = dfPAPivot.groupby(_keyColumns + ['A','IB_OB','SEGID'],as_index=False).agg(D_BRD=('D_BRDA','sum'),W_BRD=('W_BRDA','sum'))
dfPABoardings = dfPABoardings[(dfPABoardings['D_BRD']>0) | (dfPABoardings['W_BRD']>0)]
#display(dfPABoardings)

# calculate alightings (exits)
dfPAAlightings = dfPAPivot.groupby(_keyColumns + ['B','IB_OB','SEGID'],as_index=False).agg(X_XIT=('X_XITB','sum'))
dfPAAlightings = dfPAAlightings[(dfPAAlightings['X_XIT']>0)]
#display(dfPAAlightings)

# combine
dfPABoardAlight = pd.DataFrame.merge(dfPABoardings, dfPAAlightings, left_on=(_keyColumns + ['A','IB_OB','SEGID']), right_on=(_keyColumns + ['B','IB_OB','SEGID']), how='outer')
dfPABoardAlight = dfPABoardAlight.fillna(0)
dfPABoardAlight['STOPNODE'] = dfPABoardAlight[['A', 'B']].max(axis=1).astype(int)
dfPABoardAlight = dfPABoardAlight[['scenarioID','TimeOfDay','MODE','NAME','IB_OB','SEGID','HeirarchyMode','STOPNODE','D_BRD','W_BRD','X_XIT']]
dfPABoardAlight.columns = ['s','tod','md','nm','io','segid','hmd','nid','db','wb','x']
dfPABoardAlight

,s,tod,md,nm,io,segid,hmd,nid,db,wb,x
0,1.01,OK,4,BE630,0,0,4,6548,0.0,0.45,0.11
1,1.01,OK,4,BE630,0,0,4,6888,0.0,0.15,0.00
2,1.01,OK,4,BE630,0,0,4,6923,0.0,0.01,0.00
3,1.01,OK,4,BE630,0,0,4,7055,0.0,1.44,5.44
4,1.01,OK,4,BE630,0,0,4,7095,0.0,0.24,0.21
...,...,...,...,...,...,...,...,...,...,...,...
1039588,6.18,PK,9,WE603,XX,3400_000.3,9,28823,0.0,0.00,3.98
1039589,6.18,PK,9,WE603,XX,0203_003.9,9,28894,0.0,0.00,0.05
1039590,6.18,PK,9,WE603,XX,3400_000.6,9,28899,0.0,0.00,3.07
1039591,6.18,PK,9,WE603,XX,3404_001.5,9,28935,0.0,0.00,0.80


# Export jsons

In [25]:
export = False

_outputdir_paLink        = os.path.join(pathWebData,'transitpalink'       )
_outputdir_paNode        = os.path.join(pathWebData,'transitpanode'       )
_outputdir_routeModeName = os.path.join(pathWebData,'transitroutenamemode')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        #print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        if (len(dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())>0):
            _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
            #print(_id)
            _file = _id + '.json'
            print(_file)
            _outputfile_paLink = os.path.join(_outputdir_paLink,_file)
            _outputfile_paNode = os.path.join(_outputdir_paNode,_file)
            _outputfile_routeModeName = os.path.join(_outputdir_routeModeName,_file)

            dfLinkOut = dfPARiders[dfPARiders['s']==row['scenarioID']]
            dfLinkOut = dfLinkOut.drop(columns=('s'))
            dfLinkOut = dfLinkOut.set_index('lid')
            if export: dfLinkOut.to_json(_outputfile_paLink,orient="table", indent=1) #,orient='records'

            dfNodeOut = dfPABoardAlight[dfPABoardAlight['s']==row['scenarioID']]
            dfNodeOut = dfNodeOut.drop(columns=('s'))
            dfNodeOut = dfNodeOut.set_index('nid')
            if export: dfNodeOut.to_json(_outputfile_paNode,orient="table", indent=1) #,orient='records'

            # Output routename list per scenario
            dfRouteNamesOut = dfLinkOut[['nm','md']].drop_duplicates()
            dfRouteNamesOut = pd.DataFrame.merge(dfRouteNamesOut,dfRouteSummaryNames_Revised[['NAME_New','LONGNAME']],left_on='nm',right_on='NAME_New',how='left')
            dfRouteNamesOut = dfRouteNamesOut.drop(columns='NAME_New')
            dfRouteNamesOut = dfRouteNamesOut.rename(columns={'nm':'value','md':'mode','LONGNAME':'label'})
            dfRouteNamesOut = dfRouteNamesOut[['value','mode','label']].drop_duplicates()
            dfRouteNamesOut = dfRouteNamesOut.set_index('value')
            if export: dfRouteNamesOut.to_json(_outputfile_routeModeName,orient="table", indent=1) #,orient='records'
            
            #display(dfTemp)
    except:
        print('scenarioID ' + str(row['scenarioID']) + ' error')
        continue

id_23888985563cb5d49295ba5.99558740.json
id_63298618163cb5aa9ba1549.90065432.json
id_199631216463cb5b3a4cfcb1.37627729.json
id_98449779163cadd267afbd0.97795935.json
id_72075547863fe263fdc2889.13328783.json
id_203185003163fe2719e2fec6.80974946.json
id_37001244863cadd40e1b8f7.52642706.json
id_119707969463fe253a3aeac3.60917424.json
id_18881358963fe26b786e365.87703358.json
id_72534583163cb54226ea202.98630416.json
id_76121543563fe255414a902.24461086.json
id_126457316463fe26c83bff30.30615209.json
id_133633625463cb540774f8a1.20476248.json
id_55521217063fe256f1dc401.11178986.json
id_94673840963fe26e2dbfcd3.30857457.json
id_73730688563cb5d2222cdc4.25550859.json
id_164525944263cb5a65b998b7.54218474.json
id_55480621263cb5b14c32686.99056477.json
id_144890492163cb543f110d71.15198462.json
id_206030483863fe2605c50b99.31226111.json
id_76869800863fe26fb3bb595.01461071.json
id_71963308763cb5d35348012.40299726.json
id_22075304363cb5a8a520834.45828991.json
id_161746222663cb5b26a7d924.02056464.json
id_1612

# Export Link and Node Shapefiles

In [26]:
export = False

if export:

        
    # Import the required ArcGIS API for Python modules
    import pandas as pd

    import arcgis
    gis = arcgis.GIS()

    sdfLink = pd.DataFrame.spatial.from_featureclass(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-02-02 - Link.shp")
    sdfNode = pd.DataFrame.spatial.from_featureclass(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-02-02 - Node.shp")

    # scrape out unneeded data
    sdfLink = sdfLink[['LINKID','SHAPE']]
    sdfNode = sdfNode[['N','SHAPE']]

    # used links export
    sdfLinksForTransit = pd.DataFrame.merge(sdfLink,dfPARiders[['lid']].drop_duplicates(),left_on='LINKID',right_on='lid',how='inner')
    sdfLinksForTransit.drop(columns=('lid'),inplace=True)
    sdfLinksForTransit.spatial.to_featureclass(os.path.join('_tdm_network','transitlinks.shp'),sanitize_columns=False)

    # used nodes export
    sdfNodesForTransit = pd.DataFrame.merge(sdfNode,dfPABoardAlight[['nid']].drop_duplicates(),left_on='N',right_on='nid',how='inner')
    sdfNodesForTransit.drop(columns=('nid'),inplace=True)
    sdfNodesForTransit.spatial.to_featureclass(os.path.join('_tdm_network','transitnodes.shp'),sanitize_columns=False)